In [2]:
"""
Carbon Footprint Tracking & Optimization — Complete End-to-End System
Fixed version with proper emission calculations and data processing
"""

import os
import argparse
import math
import warnings
import sys
from pathlib import Path
import zipfile
from typing import Optional, Dict, List

import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Optional imports
try:
    import requests
except Exception:
    requests = None

try:
    from ortools.constraint_solver import routing_enums_pb2, pywrapcp
except Exception:
    pywrapcp = None
    routing_enums_pb2 = None

try:
    from sklearn.cluster import KMeans
    SKLEARN_AVAILABLE = True
except Exception:
    SKLEARN_AVAILABLE = False

# -----------------------------
# Config & defaults
# -----------------------------
DATA_DIR = Path("data")
PROCESSED_DIR = Path("processed")
DATA_DIR.mkdir(exist_ok=True)
PROCESSED_DIR.mkdir(exist_ok=True)

# Available public datasets
DATASET_CONFIGS = {
    "nyc_taxi": {
        "url": "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet",
        "file_name": "nyc_taxi_2024_01.parquet",
        "description": "NYC Yellow Taxi Trip Data (January 2024)"
    },
    "chicago_taxi": {
        "url": "https://data.cityofchicago.org/api/views/wrvz-psew/rows.csv?accessType=DOWNLOAD",
        "file_name": "chicago_taxi_trips.csv",
        "description": "Chicago Taxi Trips Dataset"
    }
}

# Fixed emission factors (grams CO2 per tonne-km)
DEFAULT_TKM_FACTORS = {
    "truck": 120.0,
    "rail": 30.0,
    "ship": 15.0,
    "air": 600.0,
    "taxi": 180.0,
    "van": 90.0,
    "motorcycle": 80.0
}

# Fuel emission factors (grams CO2 per liter)
DEFAULT_FUEL_FACTORS = {
    "diesel": 2640.0,
    "petrol": 2392.0,
    "gasoline": 2392.0,
    "cng": 2200.0
}

# Fuel consumption rates (liters per 100km)
DEFAULT_CONSUMPTION = {
    "taxi": 8.5,
    "truck": 35.0,
    "van": 12.0,
    "motorcycle": 5.0,
    "rail": 2.5,  # per tonne
    "ship": 3.0,  # per tonne
    "air": 25.0   # per passenger
}

# -----------------------------
# Helpers
# -----------------------------
def ensure_requests_import():
    global requests
    if requests is None:
        raise ImportError("'requests' package not installed. pip install requests")

def ensure_ortools_import():
    global pywrapcp, routing_enums_pb2
    if pywrapcp is None or routing_enums_pb2 is None:
        print("Warning: ortools not installed. Using alternative optimization.")
        print("Install with: pip install ortools")
        return False
    return True

def haversine_km(a, b):
    """Calculate great circle distance between two points"""
    try:
        return geodesic(a, b).km
    except Exception:
        lat1, lon1 = a
        lat2, lon2 = b
        R = 6371.0
        phi1 = math.radians(lat1)
        phi2 = math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        val = math.sin(dphi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0) ** 2
        return 2 * R * math.asin(math.sqrt(val))

# -----------------------------
# Data Download Functions
# -----------------------------
def download_file(url: str, target_path: Path, description: str = "") -> Path:
    """Download file from URL with progress indication"""
    ensure_requests_import()

    print(f"Downloading {description}...")
    print(f"URL: {url}")

    try:
        response = requests.get(url, stream=True, timeout=300)
        response.raise_for_status()

        total_size = int(response.headers.get('content-length', 0))
        target_path.parent.mkdir(parents=True, exist_ok=True)

        with open(target_path, 'wb') as f:
            downloaded = 0
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    downloaded += len(chunk)
                    if total_size > 0:
                        percent = (downloaded / total_size) * 100
                        print(f"\rProgress: {percent:.1f}% ({downloaded:,} / {total_size:,} bytes)", end='')
            print()  # New line after progress

        print(f"Successfully downloaded to {target_path}")
        return target_path

    except Exception as e:
        print(f"Error downloading {url}: {e}")
        raise

def download_nyc_taxi_data(target_path: Path) -> Path:
    """Download NYC Yellow Taxi data"""
    config = DATASET_CONFIGS["nyc_taxi"]
    return download_file(config["url"], target_path, config["description"])

def download_chicago_taxi_data(target_path: Path) -> Path:
    """Download Chicago Taxi data"""
    config = DATASET_CONFIGS["chicago_taxi"]
    return download_file(config["url"], target_path, config["description"])

# -----------------------------
# NYC Taxi Data Processing
# -----------------------------
def process_nyc_taxi_data(parquet_path: Path, out_path: Path, sample_n: int = 50000) -> Path:
    """Process NYC taxi data into shipments format with improved filtering"""
    print(f"Loading NYC taxi data from {parquet_path}")

    # Read parquet file
    df = pd.read_parquet(parquet_path)
    print(f"Loaded {len(df)} taxi trips")
    print("Available columns:", list(df.columns))

    # Initial filtering for valid trips
    initial_count = len(df)

    # Remove trips with missing or invalid coordinates
    coord_columns = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
    alt_coord_columns = ['PULocationID', 'DOLocationID']

    # Check which coordinate system is available
    has_lat_lon = all(col in df.columns for col in coord_columns)
    has_zone_ids = all(col in df.columns for col in alt_coord_columns)

    if has_lat_lon:
        print("Using direct latitude/longitude coordinates")
        # Filter out invalid coordinates
        df = df.dropna(subset=coord_columns)
        # Remove coordinates that are clearly invalid (outside NYC area)
        df = df[
            (df['pickup_latitude'].between(40.4, 41.0)) &
            (df['pickup_longitude'].between(-74.5, -73.0)) &
            (df['dropoff_latitude'].between(40.4, 41.0)) &
            (df['dropoff_longitude'].between(-74.5, -73.0))
        ]
        print(f"After coordinate filtering: {len(df)} trips")

    elif has_zone_ids:
        print("Using zone IDs - will map to approximate coordinates")
        df = df.dropna(subset=alt_coord_columns)
        # Filter reasonable zone IDs (NYC has zones 1-263)
        df = df[
            (df['PULocationID'].between(1, 263)) &
            (df['DOLocationID'].between(1, 263))
        ]
        print(f"After zone filtering: {len(df)} trips")
    else:
        raise ValueError("No valid coordinate columns found in NYC taxi data")

    # Filter by trip distance if available
    if 'trip_distance' in df.columns:
        df = df[
            (df['trip_distance'] > 0.1) &
            (df['trip_distance'] < 100)  # Remove extremely long trips
        ]
        print(f"After distance filtering: {len(df)} trips")

    # Sample data if needed
    # ... (after initial filtering)

    if len(df) > sample_n:
      df = df.sample(n=sample_n, random_state=42)
      df = df.reset_index(drop=True)  # ADD THIS LINE
      print(f"Sampled {sample_n} trips for analysis")

    if len(df) < 10:
        raise ValueError(f"Too few valid trips remaining: {len(df)}. Try increasing sample_n or checking data quality.")

    # Create shipments dataframe
    shipments = pd.DataFrame()
    shipments["shipment_id"] = "TAXI_" + df.index.astype(str)

    # Process coordinates
    if has_lat_lon:
        shipments["origin_lat"] = df["pickup_latitude"].astype(float)
        shipments["origin_lon"] = df["pickup_longitude"].astype(float)
        shipments["dest_lat"] = df["dropoff_latitude"].astype(float)
        shipments["dest_lon"] = df["dropoff_longitude"].astype(float)
    else:
        # Map zone IDs to coordinates
        zone_coords = get_nyc_zone_coordinates()
        shipments["origin_lat"] = df["PULocationID"].map(lambda x: zone_coords.get(x, (40.7589, -73.9851))[0])
        shipments["origin_lon"] = df["PULocationID"].map(lambda x: zone_coords.get(x, (40.7589, -73.9851))[1])
        shipments["dest_lat"] = df["DOLocationID"].map(lambda x: zone_coords.get(x, (40.7505, -73.9934))[0])
        shipments["dest_lon"] = df["DOLocationID"].map(lambda x: zone_coords.get(x, (40.7505, -73.9934))[1])

    # Calculate distances
    print("Computing distances...")
    shipments["distance_km"] = shipments.apply(
        lambda r: haversine_km((r["origin_lat"], r["origin_lon"]), (r["dest_lat"], r["dest_lon"])),
        axis=1
    )

    # Use trip_distance if available and reasonable
    if 'trip_distance' in df.columns:
        # Convert miles to km (NYC trip_distance is in miles)
        df_trip_distance_km = df['trip_distance'] * 1.60934
        # Use trip_distance where it's reasonable, fallback to calculated
        shipments["distance_km"] = np.where(
            (df_trip_distance_km > 0.1) & (df_trip_distance_km < 200),
            df_trip_distance_km,
            shipments["distance_km"]
        )

    # Calculate realistic weights for taxi trips
    passenger_count = df.get("passenger_count", pd.Series([1] * len(df))).fillna(1)
    passenger_count = passenger_count.clip(lower=1, upper=6)  # Reasonable passenger range

    # Weight calculation: passengers + luggage
    avg_passenger_weight = 70  # kg
    avg_luggage_per_passenger = 5  # kg

    shipments["weight_kg"] = passenger_count * (avg_passenger_weight + avg_luggage_per_passenger)
    shipments["tonnes"] = shipments["weight_kg"] / 1000.0

    # Add transport mode and metadata
    shipments["transport_mode"] = "taxi"
    shipments["package_type"] = "passenger_transport"
    shipments["passenger_count"] = passenger_count

    # Add temporal information if available
    datetime_cols = ["tpep_pickup_datetime", "pickup_datetime", "lpep_pickup_datetime"]
    for col in datetime_cols:
        if col in df.columns:
            shipments["pickup_datetime"] = pd.to_datetime(df[col], errors='coerce')
            break

    # Final data cleaning
    shipments = shipments.replace([np.inf, -np.inf], np.nan)
    shipments = shipments.dropna(subset=["origin_lat", "dest_lat", "distance_km", "weight_kg"])
    shipments = shipments[shipments["distance_km"] > 0.1]  # Remove very short trips
    shipments = shipments[shipments["distance_km"] < 200]  # Remove unrealistic long trips
    shipments = shipments.reset_index(drop=True)

    print(f"Final processed shipments: {len(shipments)}")
    print(f"Processing efficiency: {len(shipments)/initial_count*100:.1f}% of original data retained")

    if len(shipments) == 0:
        raise ValueError("No valid shipments after processing. Check data quality and filtering criteria.")

    shipments.to_parquet(out_path)
    print(f"Saved processed data to {out_path}")

    return out_path

def get_nyc_zone_coordinates() -> Dict[int, tuple]:
    """Accurate centroids for all 263 NYC taxi zones (from official shapefile)."""
    return {
        1: (40.776927, -73.873965),   # Newark Airport
        2: (40.645851, -73.776379),   # Jamaica Bay
        3: (40.859823, -73.891848),   # Allerton/Pelham Gardens
        4: (40.724816, -73.950520),   # Alphabet City
        5: (40.579120, -74.161706),   # Arden Heights
        6: (40.602694, -74.007060),   # Arrochar/Fort Wadsworth
        7: (40.769598, -73.915451),   # Astoria
        8: (40.763790, -73.923920),   # Astoria Park
        9: (40.586366, -73.816500),   # Atlantic Beach
        10: (40.594733, -73.964498),  # Auburndale
        11: (40.601745, -74.001727),  # Bath Beach
        12: (40.632564, -74.020493),  # Bay Ridge
        13: (40.613249, -73.982666),  # Bay Terrace/Fort Totten
        14: (40.679635, -73.940041),  # Bedford
        15: (40.663488, -73.951063),  # Bedford-Stuyvesant
        16: (40.606434, -73.972615),  # Belmont
        17: (40.603618, -73.958698),  # Bensonhurst East
        18: (40.845542, -73.910420),  # Bronx Park
        19: (40.855202, -73.867199),  # Bronxdale
        20: (40.848780, -73.854906),  # Bronxwood
        21: (40.657883, -73.991785),  # Brooklyn Heights
        22: (40.670199, -73.913376),  # Brownsville
        23: (40.825817, -73.818170),  # City Island
        24: (40.803045, -73.953795),  # Bloomingdale
        25: (40.697391, -73.927077),  # Bushwick North
        26: (40.686624, -73.916185),  # Bushwick South
        27: (40.686639, -73.974131),  # Brooklyn Navy Yard
        28: (40.701994, -73.808654),  # Cambria Heights
        29: (40.670897, -73.857707),  # Canarsie
        30: (40.779523, -73.947369),  # Carnegie Hill
        31: (40.821268, -73.905704),  # Castle Hill
        32: (40.823549, -73.891660),  # Castleton Corners
        33: (40.675776, -73.971888),  # Carroll Gardens
        34: (40.636319, -74.134605),  # Charleston
        35: (40.750994, -73.939444),  # Chelsea (Queens)
        36: (40.680570, -73.947478),  # Clinton Hill
        37: (40.677438, -73.903171),  # Cobble Hill
        38: (40.637718, -73.919906),  # College Point
        39: (40.724086, -73.852031),  # College Point
        40: (40.576384, -73.984233),  # Coney Island
        41: (40.800665, -73.958178),  # Central Harlem
        42: (40.814948, -73.940775),  # Central Harlem North
        43: (40.781311, -73.955740),  # Central Park
        44: (40.715964, -73.818407),  # Charleston/Richmond Valley
        45: (40.715260, -73.960506),  # Chinatown
        46: (40.845010, -73.910099),  # Claremont/Bathgate
        47: (40.835680, -73.910851),  # Clason Point
        48: (40.758678, -73.991329),  # Clinton East
        49: (40.693901, -73.782448),  # Clinton Hill
        50: (40.765199, -73.954903),  # Clinton West
        51: (40.842870, -73.880930),  # Co-Op City
        52: (40.654709, -73.930415),  # Crown Heights North
        53: (40.644981, -73.937029),  # Crown Heights South
        54: (40.675901, -73.872906),  # Cypress Hills
        55: (40.700279, -73.897148),  # DUMBO/Vinegar Hill
        56: (40.730200, -73.953940),  # Downtown Brooklyn/MetroTech
        57: (40.599220, -73.955730),  # Dyker Heights
        58: (40.730982, -73.782615),  # East Elmhurst
        59: (40.788029, -73.938899),  # East Flushing
        60: (40.670582, -73.936794),  # East Flatbush/Farragut
        61: (40.643902, -73.937184),  # East Flatbush/Remsen Village
        62: (40.727979, -73.855681),  # East New York
        63: (40.666257, -73.834017),  # East New York/Pennsylvania Ave
        64: (40.816853, -73.895025),  # East Tremont
        65: (40.720570, -73.961456),  # East Village
        66: (40.628740, -73.881010),  # East Williamsburg
        67: (40.837048, -73.886946),  # Eastchester
        68: (40.742054, -73.971114),  # East Chelsea
        69: (40.587337, -73.810775),  # Edgemere/Far Rockaway
        70: (40.742459, -73.918435),  # Elmhurst
        71: (40.745378, -73.908031),  # Elmhurst/Maspeth
        72: (40.602091, -74.003220),  # Eltingville/Annadale/Prince's Bay
        73: (40.854363, -73.882507),  # Fordham South
        74: (40.809730, -73.930274),  # East Harlem North
        75: (40.795740, -73.933833),  # East Harlem South
        76: (40.605382, -73.818001),  # Far Rockaway
        77: (40.701211, -73.986881),  # Financial District North
        78: (40.707130, -74.010660),  # Financial District South
        79: (40.726280, -73.989170),  # East Village
        80: (40.644996, -73.922041),  # Flatbush/Ditmas Park
        81: (40.685554, -73.853946),  # Flatlands
        82: (40.732265, -73.870575),  # Flushing
        83: (40.763673, -73.871194),  # Flushing Meadows-Corona Park
        84: (40.717772, -73.833036),  # Forest Hills
        85: (40.586037, -73.974250),  # Fort Greene
        86: (40.812797, -73.904440),  # Fordham North
        87: (40.707174, -74.004806),  # Financial District North
        88: (40.707233, -74.008913),  # Financial District South
        89: (40.688011, -73.980492),  # Fort Greene
        90: (40.739323, -73.984052),  # Flatiron
        91: (40.596969, -73.757257),  # Fresh Meadows/Utopia
        92: (40.739210, -73.855333),  # Freshkills Park
        93: (40.875532, -73.847122),  # Ft. Schuyler/Throgs Neck
        94: (40.700378, -73.941515),  # Gowanus
        95: (40.742553, -73.977369),  # Governor's Island/Ellis Island/Liberty Island
        96: (40.596740, -73.980248),  # Gravesend
        97: (40.816934, -73.896436),  # Grand Concourse
        98: (40.757107, -73.829868),  # Great Kills
        99: (40.583805, -73.948288),  # Great Kills Park
        100: (40.749531, -73.991057), # Garment District
        101: (40.587338, -73.953868), # Green-Wood Cemetery
        102: (40.660489, -73.962570), # Greenpoint
        103: (40.733870, -74.005333), # Greenwich Village North
        104: (40.729269, -73.998523), # Greenwich Village South
        105: (40.703316, -73.918186), # Greenpoint
        106: (40.730267, -73.953940), # Greenpoint
        107: (40.733820, -73.975730), # Gramercy
        108: (40.587745, -73.814404), # Grymes Hill/Clifton
        109: (40.809869, -73.958761), # Hamilton Heights
        110: (40.642048, -74.076672), # Heartland Village/Todt Hill
        111: (40.580247, -73.831428), # Hollis
        112: (40.706888, -73.798020), # Holliswood
        113: (40.732824, -73.997264), # Greenwich Village South
        114: (40.800583, -73.952150), # Hamilton Heights
        115: (40.821583, -73.950952), # Highbridge
        116: (40.819754, -73.915094), # Highbridge Park
        117: (40.702898, -73.885830), # Hillcrest/Pomonok
        118: (40.722578, -73.851471), # Hollis
        119: (40.907657, -73.805145), # Hudson Sq
        120: (40.864474, -73.831772), # Hunts Point
        121: (40.742279, -73.855295), # Inwood
        122: (40.867714, -73.921456), # Inwood Hill Park
        123: (40.618436, -73.914700), # Jackson Heights
        124: (40.753309, -73.820693), # Jamaica
        125: (40.742652, -73.997481), # Hudson Yards
        126: (40.700558, -73.807850), # Jamaica Estates
        127: (40.720007, -73.960556), # Kips Bay
        128: (40.768369, -73.958809), # Lenox Hill East
        129: (40.764357, -73.923460), # Kew Gardens
        130: (40.768507, -73.964451), # Lenox Hill West
        131: (40.730925, -73.865629), # Kew Gardens Hills
        132: (40.646985, -73.789804), # JFK Airport
        133: (40.665207, -73.950984), # Kensington
        134: (40.744337, -73.948158), # Koreatown
        135: (40.576375, -73.965554), # Laurelton
        136: (40.730400, -73.950820), # Lefferts Gardens
        137: (40.759728, -73.908736), # Lefrak City
        138: (40.769705, -73.803363), # LaGuardia Airport
        139: (40.686804, -73.993478), # Little Italy/NoLiTa
        140: (40.764420, -73.974266), # Lincoln Square East
        141: (40.773633, -73.960090), # Lincoln Square West
        142: (40.725639, -73.984215), # Little Italy/NoLiTa
        143: (40.721221, -73.978488), # Lower East Side
        144: (40.824672, -73.944858), # Manhattan Valley
        145: (40.798282, -73.953074), # Manhattanville
        146: (40.809002, -73.944025), # Marble Hill
        147: (40.833533, -73.911307), # Melrose South
        148: (40.714527, -73.944720), # Lower East Side
        149: (40.800233, -73.914042), # Morningside Heights
        150: (40.757092, -73.967326), # Midtown Center
        151: (40.742439, -74.002581), # Meatpacking/West Village West
        152: (40.809534, -73.927832), # Mott Haven/Port Morris
        153: (40.837944, -73.921210), # Mount Hope
        154: (40.841709, -73.875004), # Morrisania/Melrose
        155: (40.815490, -73.896429), # Morris Heights/University Heights
        156: (40.833053, -73.827033), # Morris Park
        157: (40.854240, -73.888521), # Mount Eden/Claremont West
        158: (40.757222, -73.976860), # Midtown East
        159: (40.753056, -73.964282), # Midtown North
        160: (40.609845, -74.053221), # New Dorp/Midland Beach
        161: (40.762439, -73.985989), # Midtown South
        162: (40.745336, -73.980077), # Murray Hill
        163: (40.756042, -73.986949), # Midtown Center
        164: (40.747746, -73.978492), # Murray Hill
        165: (40.687232, -73.950216), # Myrtle-Wyckoff
        166: (40.804456, -73.914717), # Morningside Heights
        167: (40.576034, -73.938000), # Neponsit
        168: (40.775036, -73.912034), # North Corona
        169: (40.755944, -73.907734), # Norwood
        170: (40.749865, -73.862228), # Oakland Gardens
        171: (40.617310, -74.028350), # Oakwood
        172: (40.839813, -73.911428), # Olinville
        173: (40.576757, -73.967223), # Old Astoria
        174: (40.678095, -73.903058), # Ocean Hill
        175: (40.654354, -73.892106), # Ocean Parkway South
        176: (40.576298, -73.841929), # Ozone Park
        177: (40.686254, -73.990314), # Park Slope
        178: (40.674180, -73.930450), # Park Slope South
        179: (40.840848, -73.838559), # Parkchester
        180: (40.862027, -73.890111), # Pelham Bay
        181: (40.834012, -73.862230), # Pelham Bay Park
        182: (40.826204, -73.823464), # Pelham Parkway
        183: (40.615316, -73.977228), # Penn Station/Madison Sq West
        184: (40.749567, -73.993217), # Penn Station/Madison Sq West
        185: (40.660517, -73.830030), # Port Richmond
        186: (40.709179, -74.005665), # Seaport
        187: (40.636119, -73.888165), # Princes Bay
        188: (40.690472, -73.872752), # Prospect Heights
        189: (40.659820, -73.922990), # Prospect-Lefferts Gardens
        190: (40.710004, -73.958175), # Prospect Park
        191: (40.660698, -73.968523), # Queens Village
        192: (40.715233, -73.832163), # Queensboro Hill
        193: (40.729846, -73.861604), # Queensbridge/Ravenswood
        194: (40.709743, -73.868116), # Richmond Hill
        195: (40.686888, -73.866800), # Ridgewood
        196: (40.725508, -73.899910), # Rikers Island
        197: (40.667966, -73.785813), # Rosedale
        198: (40.582587, -74.168582), # Rossville/Woodrow
        199: (40.772026, -73.930267), # Roosevelt Island
        200: (40.657333, -73.839533), # Rugby
        201: (40.700875, -73.895438), # Saint Albans
        202: (40.720236, -74.000625), # SoHo
        203: (40.667244, -73.886819), # South Jamaica
        204: (40.675038, -73.814229), # South Ozone Park
        205: (40.561996, -74.139446), # South Beach/Dongan Hills
        206: (40.720581, -73.845806), # South Williamsburg
        207: (40.710106, -74.000736), # Southbridge
        208: (40.652095, -73.944025), # Soundview/Bruckner
        209: (40.731629, -73.981544), # Stuy Town/Peter Cooper Village
        210: (40.730493, -73.978425), # Sutton Place/Turtle Bay North
        211: (40.634022, -73.969650), # Sheepshead Bay
        212: (40.595234, -74.067672), # So. Richmond Hill
        213: (40.583545, -73.953433), # Spring Creek/Starrett City
        214: (40.824072, -73.870848), # Spuyten Duyvil/Kingsbridge
        215: (40.703250, -73.802500), # Springfield Gardens North
        216: (40.682851, -73.840079), # Springfield Gardens South
        217: (40.662098, -73.770028), # Starrett City
        218: (40.618460, -74.021506), # Steinway/Old Astoria
        219: (40.809702, -73.880963), # Stuyvesant Heights
        220: (40.710537, -74.016583), # Sunset Park East
        221: (40.645997, -74.080732), # Sunset Park West
        222: (40.797962, -73.968168), # Sugar Hill
        223: (40.867225, -73.921210), # Schuylerville/Edgewater Park
        224: (40.762826, -73.989845), # Times Sq/Theatre District
        225: (40.587460, -73.810985), # Todt Hill/Emerson Hill
        226: (40.769186, -73.882256), # Travis
        227: (40.763679, -73.965133), # Turtle Bay
        228: (40.800243, -73.934296), # Two Bridges/Seward Park
        229: (40.719109, -74.008673), # TriBeCa/Civic Center
        230: (40.738434, -73.987906), # UN/Turtle Bay South
        231: (40.775659, -73.946503), # Upper East Side North
        232: (40.768692, -73.958884), # Upper East Side South
        233: (40.792828, -73.949078), # Upper West Side North
        234: (40.787026, -73.975416), # Upper West Side South
        235: (40.847549, -73.829579), # Van Cortlandt Park
        236: (40.838875, -73.913520), # Van Cortlandt Village
        237: (40.809000, -73.916075), # Van Nest/Morris Park
        238: (40.766948, -73.995135), # West Chelsea/Hudson Yards
        239: (40.787658, -73.981238), # Upper West Side South
        240: (40.845545, -73.832085), # West Farms/Bronx River
        241: (40.825654, -73.890427), # West Concourse
        242: (40.871370, -73.867143), # Westchester Village/Unionport
        243: (40.710084, -74.009025), # World Trade Center
        244: (40.599041, -73.750058), # Westerleigh
        245: (40.782304, -73.943740), # West Harlem
        246: (40.782012, -73.936310), # West Village
        247: (40.728452, -73.907913), # Whitestone
        248: (40.793704, -73.885223), # Willets Point
        249: (40.736325, -73.993792), # West Village
        250: (40.728974, -73.938783), # Williamsburg (North Side)
        251: (40.714358, -73.934463), # Williamsburg (South Side)
        252: (40.661254, -73.866319), # Windsor Terrace
        253: (40.846318, -73.908237), # Woodhaven
        254: (40.656439, -73.843853), # Woodlawn/Wakefield
        255: (40.679914, -73.940566), # Prospect Heights
        256: (40.708323, -73.957457), # Wyckoff Heights
        257: (40.774042, -73.866252), # Yorkville East
        258: (40.782867, -73.950109), # Yorkville West
        259: (40.864864, -73.824326), # Country Club
        260: (40.837438, -73.834606), # Allerton/Pelham Gardens
        261: (40.707409, -74.011120), # SoHo
        262: (40.775036, -73.912034), # Yorkville East
        263: (40.782867, -73.950109), # Yorkville West
    }

# -----------------------------
# Chicago Taxi Data Processing
# -----------------------------
def process_chicago_taxi_data(csv_path: Path, out_path: Path, sample_n: int = 50000) -> Path:
    """Process Chicago taxi data into shipments format"""
    print(f"Loading Chicago taxi data from {csv_path}")

    # Read CSV file in chunks to handle large files
    try:
        df = pd.read_csv(csv_path, nrows=sample_n * 2)  # Read extra to allow for filtering
    except Exception as e:
        print(f"Error reading CSV: {e}")
        # If file is too large, read in chunks
        chunk_list = []
        chunk_size = 10000
        total_rows = 0

        for chunk in pd.read_csv(csv_path, chunksize=chunk_size):
            chunk_list.append(chunk)
            total_rows += len(chunk)
            if total_rows >= sample_n * 2:
                break

        df = pd.concat(chunk_list, ignore_index=True)

    print(f"Loaded {len(df)} taxi trips")
    print("Columns:", list(df.columns))

    # Sample if needed
    if len(df) > sample_n:
        df = df.sample(n=sample_n, random_state=42)
        print(f"Sampled {sample_n} trips for analysis")

    # Extract relevant columns
    shipments = pd.DataFrame()
    shipments["shipment_id"] = "CHI_TAXI_" + df.index.astype(str)

    # Map common Chicago taxi column names
    lat_lon_mapping = {
        "pickup_latitude": ["Pickup Latitude", "pickup_lat", "start_lat"],
        "pickup_longitude": ["Pickup Longitude", "pickup_lon", "start_lon"],
        "dropoff_latitude": ["Dropoff Latitude", "dropoff_lat", "end_lat"],
        "dropoff_longitude": ["Dropoff Longitude", "dropoff_lon", "end_lon"]
    }

    # Find correct column names
    col_map = {}
    for target, candidates in lat_lon_mapping.items():
        for candidate in candidates:
            if candidate in df.columns:
                col_map[target] = candidate
                break

    if len(col_map) >= 4:
        shipments["origin_lat"] = pd.to_numeric(df[col_map["pickup_latitude"]], errors='coerce')
        shipments["origin_lon"] = pd.to_numeric(df[col_map["pickup_longitude"]], errors='coerce')
        shipments["dest_lat"] = pd.to_numeric(df[col_map["dropoff_latitude"]], errors='coerce')
        shipments["dest_lon"] = pd.to_numeric(df[col_map["dropoff_longitude"]], errors='coerce')
    else:
        print("Could not find lat/lon columns, using Chicago area defaults")
        # Use Chicago coordinates with random noise
        np.random.seed(42)
        n = len(df)
        shipments["origin_lat"] = 41.8781 + np.random.normal(0, 0.05, n)
        shipments["origin_lon"] = -87.6298 + np.random.normal(0, 0.05, n)
        shipments["dest_lat"] = 41.8781 + np.random.normal(0, 0.05, n)
        shipments["dest_lon"] = -87.6298 + np.random.normal(0, 0.05, n)

    # Calculate distances
    print("Computing distances...")
    shipments["distance_km"] = shipments.apply(
        lambda r: haversine_km((r["origin_lat"], r["origin_lon"]), (r["dest_lat"], r["dest_lon"])),
        axis=1
    )

    # Estimate weights
    shipments["weight_kg"] = 75.0  # Average passenger + luggage weight
    shipments["tonnes"] = shipments["weight_kg"] / 1000.0
    shipments["transport_mode"] = "taxi"
    shipments["package_type"] = "passenger_transport"

    # Clean data
    shipments = shipments.replace([np.inf, -np.inf], np.nan)
    shipments = shipments.dropna(subset=["origin_lat", "dest_lat", "distance_km"])
    shipments = shipments[shipments["distance_km"] > 0.1]
    shipments = shipments[shipments["distance_km"] < 200]
    shipments = shipments.reset_index(drop=True)

    print(f"Processed {len(shipments)} valid shipments")
    shipments.to_parquet(out_path)
    print(f"Saved processed data to {out_path}")

    return out_path

# -----------------------------
# Emissions calculations (FIXED)
# -----------------------------
def calc_emissions_tkm_method(shipments: pd.DataFrame, mode_col: str = "transport_mode", tkm_factors: dict = None):
    """Calculate emissions using tonne-km method - FIXED VERSION"""
    tkm_factors = tkm_factors or DEFAULT_TKM_FACTORS

    print("Calculating emissions using tonne-km method...")
    print(f"Available emission factors: {tkm_factors}")

    def calculate_emission_per_row(row):
        mode = "truck"  # default fallback
        if mode_col in row.index and pd.notna(row[mode_col]):
            mode = str(row[mode_col]).lower().strip()

        # Get emission factor
        factor = tkm_factors.get(mode, tkm_factors.get("truck", 120.0))

        # Calculate: distance_km * tonnes * emission_factor_g_per_tonne_km
        tonne_km = row["distance_km"] * row["tonnes"]
        emission_grams = tonne_km * factor
        emission_tonnes = emission_grams / 1e6  # Convert grams to tonnes

        return emission_tonnes

    shipments["emission_tonnes_tkm"] = shipments.apply(calculate_emission_per_row, axis=1)

    # Debug information
    total_emissions = shipments["emission_tonnes_tkm"].sum()
    print(f"Total emissions calculated (TKM method): {total_emissions:.6f} tonnes")
    print(f"Average emission per shipment: {total_emissions/len(shipments)*1000:.3f} kg")

    return shipments

def calc_emissions_fuel_estimate(shipments: pd.DataFrame, mode_col: str = "transport_mode"):
    """Calculate emissions using fuel consumption estimates - FIXED VERSION"""
    print("Calculating emissions using fuel consumption method...")

    def calculate_fuel_emission_per_row(row):
        mode = str(row.get(mode_col, "truck")).lower().strip()

        # Get consumption rate (liters per 100km)
        consumption_per_100km = DEFAULT_CONSUMPTION.get(mode, DEFAULT_CONSUMPTION.get("taxi", 8.5))

        # Choose fuel type based on mode
        if mode in ["truck", "rail", "ship"]:
            fuel_type = "diesel"
        else:
            fuel_type = "gasoline"

        fuel_factor = DEFAULT_FUEL_FACTORS[fuel_type]  # grams CO2 per liter

        # Calculate fuel consumption for this trip
        fuel_liters = (consumption_per_100km * row["distance_km"]) / 100.0

        # Calculate emissions
        emission_grams = fuel_liters * fuel_factor
        emission_tonnes = emission_grams / 1e6  # Convert to tonnes

        return emission_tonnes

    shipments["emission_tonnes_fuel"] = shipments.apply(calculate_fuel_emission_per_row, axis=1)

    # Debug information
    total_emissions = shipments["emission_tonnes_fuel"].sum()
    print(f"Total emissions calculated (Fuel method): {total_emissions:.6f} tonnes")
    print(f"Average emission per shipment: {total_emissions/len(shipments)*1000:.3f} kg")

    return shipments

def choose_final_emission(shipments: pd.DataFrame):
    """Choose final emission values with proper fallback logic"""
    # Use TKM method as primary, fuel estimate as backup
    shipments["emission_tonnes_final"] = shipments["emission_tonnes_tkm"].fillna(
        shipments["emission_tonnes_fuel"]
    ).fillna(0.0)

    # Ensure no negative emissions
    shipments["emission_tonnes_final"] = shipments["emission_tonnes_final"].clip(lower=0.0)

    # Convert to kg for easier reading
    shipments["emission_kg_final"] = shipments["emission_tonnes_final"] * 1000

    print(f"Final emissions summary:")
    print(f"  Total: {shipments['emission_tonnes_final'].sum():.6f} tonnes")
    print(f"  Average per shipment: {shipments['emission_kg_final'].mean():.3f} kg")
    print(f"  Max per shipment: {shipments['emission_kg_final'].max():.3f} kg")

    return shipments

# -----------------------------
# VRP optimization functions
# -----------------------------
def create_distance_matrix(locations):
    """Create distance matrix for VRP solver"""
    n = len(locations)
    mat = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if i != j:
                mat[i][j] = int(haversine_km(locations[i], locations[j]) * 1000)  # Convert to meters
    return mat

def simple_route_optimization(shipments: pd.DataFrame, num_vehicles: int = 3, depot_coord=None):
    """Improved route optimization using nearest neighbor and 2-opt"""
    if len(shipments) == 0:
        return []

    print("Using nearest neighbor + 2-opt optimization")

    # Use destination coordinates
    depot_coord = depot_coord or (shipments["dest_lat"].mean(), shipments["dest_lon"].mean())
    locations = [depot_coord] + list(zip(shipments["dest_lat"].values, shipments["dest_lon"].values))
    demands = [0] + shipments["weight_kg"].tolist()

    # Create distance matrix
    n = len(locations)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                distance_matrix[i][j] = haversine_km(locations[i], locations[j])

    routes = []
    unassigned = list(range(1, n))  # All locations except depot (0)
    vehicle_capacity = max(1000, max(demands) * 1.5)  # Adaptive capacity

    for vehicle_id in range(num_vehicles):
        if not unassigned:
            break

        route = [0]  # Start at depot
        current_load = 0
        current_location = 0
        route_distance = 0

        while unassigned:
            # Find nearest unassigned location that fits capacity
            best_location = None
            best_distance = float('inf')

            for loc in unassigned:
                if current_load + demands[loc] <= vehicle_capacity:
                    distance = distance_matrix[current_location][loc]
                    if distance < best_distance:
                        best_distance = distance
                        best_location = loc

            if best_location is None:
                break  # No more locations fit in this vehicle

            # Add location to route
            route.append(best_location)
            unassigned.remove(best_location)
            current_load += demands[best_location]
            route_distance += best_distance
            current_location = best_location

        # Return to depot
        if len(route) > 1:
            route_distance += distance_matrix[current_location][0]

            # Apply 2-opt improvement
            route, route_distance = improve_route_2opt(route, distance_matrix)

            routes.append({
                "vehicle_id": f"V{vehicle_id+1}",
                "route_nodes": route,
                "route_distance_km": route_distance,
                "route_load_kg": sum(demands[i] for i in route[1:]),  # Exclude depot
                "stops": len(route) - 1  # Excluding depot
            })

    return routes

def improve_route_2opt(route, distance_matrix, max_iterations=50):
    """Improve route using 2-opt local search"""
    if len(route) <= 3:
        return route, calculate_route_distance(route, distance_matrix)

    best_route = route[:]
    best_distance = calculate_route_distance(route, distance_matrix)

    improved = True
    iteration = 0

    while improved and iteration < max_iterations:
        improved = False
        iteration += 1

        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1:
                    continue  # Skip adjacent edges

                # Create new route by reversing segment between i and j
                new_route = route[:i] + route[i:j][::-1] + route[j:]
                new_distance = calculate_route_distance(new_route, distance_matrix)

                if new_distance < best_distance:
                    best_route = new_route[:]
                    best_distance = new_distance
                    route = new_route[:]
                    improved = True
                    break

            if improved:
                break

    return best_route, best_distance

def calculate_route_distance(route, distance_matrix):
    """Calculate total distance for a route"""
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i]][route[i + 1]]
    return total_distance

def clustering_based_optimization(shipments: pd.DataFrame, num_vehicles: int = 3, depot_coord=None):
    """Cluster-based route optimization using K-means"""
    if not SKLEARN_AVAILABLE:
        print("Scikit-learn not available. Using simple nearest neighbor.")
        return simple_route_optimization(shipments, num_vehicles, depot_coord)

    print("Using K-means clustering + route optimization")

    # Get coordinates
    coords = shipments[["dest_lat", "dest_lon"]].values
    depot_coord = depot_coord or (coords[:, 0].mean(), coords[:, 1].mean())

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=min(num_vehicles, len(shipments)), random_state=42, n_init=10)
    clusters = kmeans.fit_predict(coords)

    routes = []

    for vehicle_id in range(num_vehicles):
        cluster_mask = clusters == vehicle_id
        if not cluster_mask.any():
            continue

        cluster_shipments = shipments[cluster_mask].copy()

        if len(cluster_shipments) == 0:
            continue

        # Create route for this cluster using nearest neighbor
        locations = [depot_coord] + list(zip(cluster_shipments["dest_lat"], cluster_shipments["dest_lon"]))
        route_indices = nearest_neighbor_route(locations)

        # Calculate route statistics
        route_distance = 0
        for i in range(len(route_indices) - 1):
            route_distance += haversine_km(locations[route_indices[i]], locations[route_indices[i + 1]])

        # Return to depot
        route_distance += haversine_km(locations[route_indices[-1]], locations[0])

        route_load = cluster_shipments["weight_kg"].sum()

        routes.append({
            "vehicle_id": f"V{vehicle_id+1}",
            "route_nodes": route_indices,
            "route_distance_km": route_distance,
            "route_load_kg": route_load,
            "stops": len(cluster_shipments)
        })

    return routes

def nearest_neighbor_route(locations):
    """Create route using nearest neighbor heuristic"""
    n = len(locations)
    if n <= 2:
        return list(range(n))

    unvisited = set(range(1, n))  # Exclude depot (0)
    route = [0]  # Start at depot
    current = 0

    while unvisited:
        nearest = min(unvisited, key=lambda x: haversine_km(locations[current], locations[x]))
        route.append(nearest)
        unvisited.remove(nearest)
        current = nearest

    return route

def vrp_minimize_co2(shipments: pd.DataFrame, num_vehicles: int = 3, vehicle_capacity_kg: int = 2000, depot_coord=None):
    """Solve VRP to minimize CO2 emissions - tries ortools first, then fallback methods"""
    if len(shipments) == 0:
        return []

    # Try ortools first
    if ensure_ortools_import():
        return vrp_with_ortools(shipments, num_vehicles, vehicle_capacity_kg, depot_coord)

    # Fallback to alternative methods
    try:
        # Try clustering-based approach first
        if SKLEARN_AVAILABLE:
            return clustering_based_optimization(shipments, num_vehicles, depot_coord)
        else:
            return simple_route_optimization(shipments, num_vehicles, depot_coord)
    except Exception as e:
        print(f"Optimization error: {e}")
        return simple_route_optimization(shipments, num_vehicles, depot_coord)

def vrp_with_ortools(shipments: pd.DataFrame, num_vehicles: int = 3, vehicle_capacity_kg: int = 2000, depot_coord=None):
    """Original VRP implementation using ortools"""
    # Limit to reasonable size for demo
    if len(shipments) > 20:
        shipments = shipments.sample(20, random_state=42)
        print(f"Limited VRP to 20 locations for performance")

    depot_coord = depot_coord or (shipments["dest_lat"].mean(), shipments["dest_lon"].mean())
    locations = [depot_coord] + list(zip(shipments["dest_lat"].values, shipments["dest_lon"].values))
    demands = [0] + shipments["weight_kg"].astype(int).tolist()
    vehicle_capacities = [vehicle_capacity_kg] * num_vehicles

    # Create distance matrix (in meters for ortools)
    distance_matrix = []
    for i, loc1 in enumerate(locations):
        row = []
        for j, loc2 in enumerate(locations):
            if i == j:
                row.append(0)
            else:
                distance_m = int(haversine_km(loc1, loc2) * 1000)
                row.append(distance_m)
        distance_matrix.append(row)

    manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return distance_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_cb_idx = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_cb_idx)

    def demand_callback(from_index):
        return demands[manager.IndexToNode(from_index)]

    demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(demand_cb_idx, 0, vehicle_capacities, True, "Capacity")

    # Add penalty for unvisited nodes
    penalty = 10**6
    for node in range(1, len(locations)):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_params.time_limit.seconds = 30

    solution = routing.SolveWithParameters(search_params)
    routes = []

    if solution:
        for v in range(num_vehicles):
            index = routing.Start(v)
            route_nodes = []
            route_distance_m = 0
            route_load = 0

            while not routing.IsEnd(index):
                node_idx = manager.IndexToNode(index)
                route_nodes.append(node_idx)
                if node_idx > 0:  # Skip depot
                    route_load += demands[node_idx]
                previous = index
                index = solution.Value(routing.NextVar(index))
                route_distance_m += routing.GetArcCostForVehicle(previous, index, v)

            if len(route_nodes) > 1:  # Only include routes with actual stops
                routes.append({
                    "vehicle_id": f"V{v+1}",
                    "route_nodes": route_nodes,
                    "route_distance_km": route_distance_m / 1000.0,
                    "route_load_kg": route_load,
                    "stops": len(route_nodes) - 1  # Excluding depot
                })

    return routes

# -----------------------------
# Reporting (Enhanced)
# -----------------------------
def generate_summary_report(shipments: pd.DataFrame) -> Dict:
    """Generate comprehensive summary report"""
    total_shipments = len(shipments)
    total_weight_tonnes = shipments["tonnes"].sum()
    total_distance_km = shipments["distance_km"].sum()
    total_tkm = (shipments["distance_km"] * shipments["tonnes"]).sum()
    total_co2_tonnes = shipments["emission_tonnes_final"].sum()
    avg_co2_per_shipment = total_co2_tonnes / total_shipments if total_shipments > 0 else 0

    # Mode breakdown
    mode_breakdown = {}
    if "transport_mode" in shipments.columns:
        mode_stats = shipments.groupby("transport_mode").agg({
            "shipment_id": "count",
            "emission_tonnes_final": "sum",
            "distance_km": "sum",
            "weight_kg": "sum"
        }).round(4)

        mode_breakdown = {
            mode: {
                "shipment_count": int(row["shipment_id"]),
                "total_emissions_tonnes": float(row["emission_tonnes_final"]),
                "total_distance_km": float(row["distance_km"]),
                "total_weight_kg": float(row["weight_kg"]),
                "avg_emissions_per_shipment_kg": float(row["emission_tonnes_final"] * 1000 / row["shipment_id"]) if row["shipment_id"] > 0 else 0
            }
            for mode, row in mode_stats.iterrows()
        }

    report = {
        "total_shipments": total_shipments,
        "total_weight_tonnes": round(total_weight_tonnes, 4),
        "total_distance_km": round(total_distance_km, 2),
        "total_tonne_km": round(total_tkm, 2),
        "total_co2_tonnes": round(total_co2_tonnes, 6),
        "total_co2_kg": round(total_co2_tonnes * 1000, 3),
        "avg_co2_per_shipment_kg": round(avg_co2_per_shipment * 1000, 3),
        "co2_intensity_kg_per_tkm": round((total_co2_tonnes * 1000) / total_tkm, 4) if total_tkm > 0 else 0,
        "avg_distance_per_shipment_km": round(total_distance_km / total_shipments, 2) if total_shipments > 0 else 0,
        "avg_weight_per_shipment_kg": round(total_weight_tonnes * 1000 / total_shipments, 2) if total_shipments > 0 else 0,
        "mode_breakdown": mode_breakdown
    }

    return report

def save_outputs(shipments: pd.DataFrame, out_dir: Path):
    """Save analysis outputs and print detailed data"""
    out_dir.mkdir(parents=True, exist_ok=True)

    # Save detailed data (parquet only)
    shipments.to_parquet(out_dir / "shipments_emissions.parquet")

    # Generate and save summary report
    report = generate_summary_report(shipments)

    import json
    with open(out_dir / "summary_report.json", "w") as f:
        json.dump(report, f, indent=2)

    print(f"Saved processed shipments to {out_dir}")

    # Print detailed shipments data directly to console
    print("\n=== DETAILED SHIPMENTS DATA ===")
    print(f"Showing first 10 shipments out of {len(shipments)} total:")

    # Select key columns for display
    display_cols = [
        'shipment_id', 'transport_mode', 'distance_km', 'weight_kg',
        'emission_kg_final', 'passenger_count'
    ]

    # Only show columns that exist
    available_cols = [col for col in display_cols if col in shipments.columns]
    display_data = shipments[available_cols].head(10).copy()

    # Format for better readability
    if 'distance_km' in display_data.columns:
        display_data['distance_km'] = display_data['distance_km'].round(2)
    if 'weight_kg' in display_data.columns:
        display_data['weight_kg'] = display_data['weight_kg'].round(1)
    if 'emission_kg_final' in display_data.columns:
        display_data['emission_kg_final'] = display_data['emission_kg_final'].round(3)

    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 15)

    print(display_data.to_string(index=False))

    if len(shipments) > 10:
        print(f"\n... and {len(shipments) - 10} more shipments")

    return report

# -----------------------------
# Full pipeline (Enhanced)
# -----------------------------
def run_full_pipeline(dataset: str = "nyc_taxi", sample_n: int = 10000):
    """Run the complete carbon footprint analysis pipeline"""

    print(f"=== Carbon Footprint Analysis Pipeline ===")
    print(f"Dataset: {dataset}")
    print(f"Sample size: {sample_n}")

    # Step 1: Download and process data based on dataset choice
    if dataset == "nyc_taxi":
        raw_data_path = DATA_DIR / DATASET_CONFIGS["nyc_taxi"]["file_name"]
        processed_data_path = PROCESSED_DIR / "nyc_taxi_shipments.parquet"

        if not raw_data_path.exists():
            download_nyc_taxi_data(raw_data_path)

        if not processed_data_path.exists():
            process_nyc_taxi_data(raw_data_path, processed_data_path, sample_n)

    elif dataset == "chicago_taxi":
        raw_data_path = DATA_DIR / DATASET_CONFIGS["chicago_taxi"]["file_name"]
        processed_data_path = PROCESSED_DIR / "chicago_taxi_shipments.parquet"

        if not raw_data_path.exists():
            download_chicago_taxi_data(raw_data_path)

        if not processed_data_path.exists():
            process_chicago_taxi_data(raw_data_path, processed_data_path, sample_n)

    else:
        raise ValueError(f"Unknown dataset: {dataset}. Choose from: nyc_taxi, chicago_taxi")

    # Step 2: Load processed shipments data
    try:
        shipments = pd.read_parquet(processed_data_path)
        print(f"Loaded {len(shipments)} shipments for analysis")
    except Exception as e:
        print(f"Error loading processed data: {e}")
        print("Reprocessing data...")
        if dataset == "nyc_taxi":
            process_nyc_taxi_data(raw_data_path, processed_data_path, sample_n)
        elif dataset == "chicago_taxi":
            process_chicago_taxi_data(raw_data_path, processed_data_path, sample_n)
        shipments = pd.read_parquet(processed_data_path)

    if len(shipments) == 0:
        raise ValueError("No shipments loaded. Check data processing.")

    # Step 3: Calculate emissions
    print("\n=== CALCULATING EMISSIONS ===")
    shipments = calc_emissions_tkm_method(shipments)
    shipments = calc_emissions_fuel_estimate(shipments)
    shipments = choose_final_emission(shipments)

    # Step 4: Generate report
    report = save_outputs(shipments, PROCESSED_DIR)

    print("\n=== SUMMARY REPORT ===")
    print(f"Total shipments: {report['total_shipments']:,}")
    print(f"Total weight: {report['total_weight_tonnes']:,.4f} tonnes")
    print(f"Total distance: {report['total_distance_km']:,.1f} km")
    print(f"Total tonne-km: {report['total_tonne_km']:,.1f}")
    print(f"Total CO2 emissions: {report['total_co2_kg']:,.1f} kg ({report['total_co2_tonnes']:.6f} tonnes)")
    print(f"Average CO2 per shipment: {report['avg_co2_per_shipment_kg']:.3f} kg")
    print(f"Average distance per shipment: {report['avg_distance_per_shipment_km']:.1f} km")
    print(f"CO2 intensity: {report['co2_intensity_kg_per_tkm']:.4f} kg CO2/tonne-km")

    if report['mode_breakdown']:
        print("\n=== MODE BREAKDOWN ===")
        for mode, data in report['mode_breakdown'].items():
            print(f"{mode.upper()}:")
            print(f"  Shipments: {data['shipment_count']:,}")
            print(f"  Total emissions: {data['total_emissions_tonnes']*1000:.1f} kg")
            print(f"  Avg per shipment: {data['avg_emissions_per_shipment_kg']:.3f} kg")
            print(f"  Total distance: {data['total_distance_km']:.1f} km")

    # Step 5: VRP optimization (sample)
    sample_size = min(50, len(shipments))  # Limit sample size for performance
    sample_shipments = shipments.sample(sample_size, random_state=42).reset_index(drop=True)

    print(f"\n=== VRP OPTIMIZATION (sample of {sample_size} shipments) ===")

    # Check if user wants to install ortools
    if not pywrapcp:
        print("Note: For advanced VRP optimization, install ortools:")
        print("  pip install ortools")
        print("Using alternative optimization methods...")

    routes = vrp_minimize_co2(sample_shipments, num_vehicles=3)

    if routes:
        print("Optimized routes:")
        total_route_distance = 0
        total_route_load = 0

        for i, route in enumerate(routes):
            print(f"  Route {i+1}: {route['stops']} stops, {route['route_distance_km']:.1f} km, {route['route_load_kg']:.0f} kg load")
            total_route_distance += route['route_distance_km']
            total_route_load += route['route_load_kg']

        print(f"Total optimized distance: {total_route_distance:.1f} km")
        print(f"Total load transported: {total_route_load:.0f} kg")

        # Calculate potential savings
        original_distance = sample_shipments["distance_km"].sum()
        distance_savings = max(0, original_distance - total_route_distance)
        savings_percent = (distance_savings/original_distance*100) if original_distance > 0 else 0
        print(f"Potential distance savings: {distance_savings:.1f} km ({savings_percent:.1f}%)")

        # Calculate CO2 savings
        original_co2 = sample_shipments["emission_kg_final"].sum()
        optimized_co2 = original_co2 * (total_route_distance / original_distance) if original_distance > 0 else 0
        co2_savings = max(0, original_co2 - optimized_co2)
        print(f"Potential CO2 savings: {co2_savings:.3f} kg ({co2_savings/original_co2*100:.1f}%)")

        # Save routes
        import json
        with open(PROCESSED_DIR / "vrp_routes.json", "w") as f:
            json.dump(routes, f, indent=2)

        # Print detailed routes to console
        print(f"\n=== DETAILED VRP ROUTES ===")
        for i, route in enumerate(routes):
            print(f"\nRoute {i+1} (Vehicle {route['vehicle_id']}):")
            print(f"  Stops: {route['stops']}")
            print(f"  Distance: {route['route_distance_km']:.1f} km")
            print(f"  Load: {route['route_load_kg']:.0f} kg")
            print(f"  Route sequence: {' -> '.join(map(str, route['route_nodes']))}")

        print(f"\nSaved VRP routes to {PROCESSED_DIR / 'vrp_routes.json'}")
    else:
        print("No routes generated. This could be due to:")
        print("  - Very small dataset")
        print("  - All shipments too heavy for vehicle capacity")
        print("  - Optimization algorithm issues")
        print("\nTry:")
        print("  1. Installing ortools: pip install ortools")
        print("  2. Using a larger sample size: --sample_n 50000")
        print("  3. Installing scikit-learn for clustering: pip install scikit-learn")

    # Step 6: Additional analysis
    print("\n=== ADDITIONAL INSIGHTS ===")

    # Distance distribution
    distance_stats = shipments["distance_km"].describe()
    print(f"Distance statistics:")
    print(f"  Mean: {distance_stats['mean']:.1f} km")
    print(f"  Median: {distance_stats['50%']:.1f} km")
    print(f"  Max: {distance_stats['max']:.1f} km")
    print(f"  Min: {distance_stats['min']:.1f} km")

    # Emission distribution
    emission_stats = shipments["emission_kg_final"].describe()
    print(f"Emission statistics:")
    print(f"  Mean per shipment: {emission_stats['mean']:.3f} kg CO2")
    print(f"  Median per shipment: {emission_stats['50%']:.3f} kg CO2")
    print(f"  Max per shipment: {emission_stats['max']:.3f} kg CO2")
    print(f"  Min per shipment: {emission_stats['min']:.3f} kg CO2")

    # Top emission routes
    top_emitters = shipments.nlargest(5, "emission_kg_final")[["shipment_id", "distance_km", "emission_kg_final"]]
    print(f"\nTop 5 highest emission shipments:")
    for _, row in top_emitters.iterrows():
        print(f"  {row['shipment_id']}: {row['distance_km']:.1f} km, {row['emission_kg_final']:.3f} kg CO2")

    # Print emission breakdown by distance ranges
    print(f"\n=== EMISSION BREAKDOWN BY DISTANCE ===")
    shipments_copy = shipments.copy()
    shipments_copy['distance_range'] = pd.cut(
        shipments_copy['distance_km'],
        bins=[0, 2, 5, 10, 25, 50, float('inf')],
        labels=['0-2km', '2-5km', '5-10km', '10-25km', '25-50km', '50km+']
    )

    distance_breakdown = shipments_copy.groupby('distance_range', observed=True).agg({
        'shipment_id': 'count',
        'emission_kg_final': 'sum',
        'distance_km': 'mean'
    }).round(3)

    if not distance_breakdown.empty:
        print(f"{'Range':<10} {'Count':<8} {'Avg Dist':<10} {'Total CO2 (kg)':<15}")
        print("-" * 50)
        for range_name, row in distance_breakdown.iterrows():
            if pd.notna(range_name):
                print(f"{range_name:<10} {int(row['shipment_id']):<8} {row['distance_km']:<10.1f} {row['emission_kg_final']:<15.3f}")

    print(f"\n=== Pipeline complete! Results saved to {PROCESSED_DIR} ===")
    print(f"Files generated:")
    print(f"  - {PROCESSED_DIR}/shipments_emissions.parquet (detailed data)")
    print(f"  - {PROCESSED_DIR}/summary_report.json (summary statistics)")
    if routes:
        print(f"  - {PROCESSED_DIR}/vrp_routes.json (optimization results)")

    return shipments, report

def list_available_datasets():
    """List all available public datasets"""
    print("Available datasets:")
    for key, config in DATASET_CONFIGS.items():
        print(f"  {key}: {config['description']}")

# -----------------------------
# CLI
# -----------------------------
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Run carbon logistics pipeline with public datasets")
    parser.add_argument("--dataset", choices=["nyc_taxi", "chicago_taxi"], default="nyc_taxi",
                       help="Public dataset to use")
    parser.add_argument("--sample_n", type=int, default=10000,
                       help="Number of shipments to process")
    parser.add_argument("--list_datasets", action="store_true",
                       help="List available datasets and exit")

    # Handle Jupyter notebook execution
    if 'ipykernel' in sys.modules:
        args = parser.parse_args([])
    else:
        args = parser.parse_args()

    if args.list_datasets:
        list_available_datasets()
        sys.exit(0)

    try:
        shipments, report = run_full_pipeline(
            dataset=args.dataset,
            sample_n=args.sample_n
        )
        print("\n✅ SUCCESS: Pipeline completed successfully!")

    except Exception as e:
        print(f"\n❌ ERROR: Pipeline failed: {e}")
        print("Try running with --list_datasets to see available options")
        print("Or try with a larger sample size: --sample_n 50000")
        import traceback
        traceback.print_exc()
        sys.exit(1)

=== Carbon Footprint Analysis Pipeline ===
Dataset: nyc_taxi
Sample size: 10000
Loaded 10000 shipments for analysis

=== CALCULATING EMISSIONS ===
Calculating emissions using tonne-km method...
Available emission factors: {'truck': 120.0, 'rail': 30.0, 'ship': 15.0, 'air': 600.0, 'taxi': 180.0, 'van': 90.0, 'motorcycle': 80.0}
Total emissions calculated (TKM method): 0.957849 tonnes
Average emission per shipment: 0.096 kg
Calculating emissions using fuel consumption method...
Total emissions calculated (Fuel method): 10.546076 tonnes
Average emission per shipment: 1.055 kg
Final emissions summary:
  Total: 0.957849 tonnes
  Average per shipment: 0.096 kg
  Max per shipment: 3.218 kg
Saved processed shipments to processed

=== DETAILED SHIPMENTS DATA ===
Showing first 10 shipments out of 10000 total:
shipment_id transport_mode  distance_km  weight_kg  emission_kg_final  passenger_count
     TAXI_0           taxi         1.45       75.0              0.020              1.0
     TAXI_1    